In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [2]:
sys.path.append(r"../../src/collectcube")
#from db import *
from db_post import *

In [3]:
#out_dir = '../../data/LUCinLA_paraguay/smSamp'
#local_db_path = os.path.join(out_dir, 'landcover.db')
out_dir = 'C:/GISprojects/ParaguayValidation/smDistricts'
local_db_path = 'C:/GISprojects/ParaguayValidation/smDistricts/landcover.db'

In [4]:
table_check = check_table(local_db_path,'pixels')

NameError: name 'check_table' is not defined

## Data cleaning (e.g. if pandas import below doesn't work)
Note: need for this should be eliminated as more formatting restrictions are placed in db entry functions

### manipulating table (should not need to do this here)

In [16]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()      
c.close()

(1994, '2701_0', 2701, 0, '2021_03-07', 40, 48, 5, '', '', '', '', '', '', '', '', '', 40, '', 60, '', '', '', '', '', '--', '--', '', 4)


### querying table (e.g. for bad records)

In [17]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification  WHERE imgDate = '2021_03-07'")
con.commit()
print(c.fetchone()) 

(1994, '2701_0', 2701, 0, '2021_03-07', 40, 48, 5, '', '', '', '', '', '', '', '', '', 40, '', 60, '', '', '', '', '', '--', '--', '', 4)


### fixing bad records

In [8]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2007-06-07' WHERE imgDate ='2007-6-07'")
con.commit()
c.close()

## Open db table as pandas dataframe

In [5]:
## This uses sqlalchemy but is prefered because it will make sure that formatting is correct
df = open_data_records_sqlachemy(local_db_path)
df.head()

,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi
0,1,0,0,2022-11-25,99,98,8.0,,,,...,,,,,,--,--,,4,0000000_0
1,2,2500,0,2007-06-07,20,31,8.0,,,,...,,,,,,--,--,,4,0002500_0
2,3,2500,0,2008-12-03,20,31,8.0,,,,...,,,,,,--,--,,4,0002500_0
3,4,2500,0,2013-07-10,20,31,8.0,,,,...,,,,,,--,--,,4,0002500_0
4,5,2500,0,2016-05-17,20,31,8.0,,,,...,,,,,,--,--,,4,0002500_0


In [6]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.head()
db_df.to_csv(os.path.join('C:/GISprojects/ParaguayValidation/smDistricts','sm_test_db.csv'), index=False)

## Get sample for target date (direct and indirect)

In [6]:
target_date = 2022

sample_type = 'area_est' ## 'area_est',  validation', 'training' 
sample_cats = 'bi_crop'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_date,local_db_path,sample_type,sample_cats,lut)
    
all_obs.head()

there are 520 observations for selected date
there are 490 center pixel observations


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi,obs_type
279,1997.0,1,0,2021-07-07,99,98,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000001_0,indirect_GE
1778,1999.0,2,0,2021-07-06,20,30,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000002_0,direct_GE
281,2003.0,4,0,2022-07-01,-1,30,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000004_0,indirect_GE
1787,2008.0,5,0,2021-07-06,20,30,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000005_0,direct_GE
1792,2013.0,8,0,2022-01-07,99,98,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000008_0,direct_GE


## Summarize sample size by sample group

In [7]:
pix = open_pix_info_sqlachemy(local_db_path)

pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
pix_data.head()

samp_sum=pix_data.groupby(['sampgroup','LC']).size().reset_index(name='counts')
samp_tot=pix_data.groupby(['sampgroup']).size().reset_index(name='total')
samp = pd.merge(samp_sum,samp_tot,on='sampgroup',how='left')
samp['per'] = samp['counts']/samp['total']
samp

,sampgroup,LC,counts,total,per
0,D0102_Concepcion_B,30,29,52,0.557692
1,D0102_Concepcion_B,98,23,52,0.442308
2,D0103_Concepcion_H,30,13,108,0.120370
3,D0103_Concepcion_H,80,28,108,0.259259
4,D0103_Concepcion_H,98,67,108,0.620370
5,D0217_SanPedro_C,30,37,69,0.536232
6,D0217_SanPedro_C,80,7,69,0.101449
7,D0217_SanPedro_C,98,25,69,0.362319
8,D0518_Caaguazu_3F,30,76,193,0.393782
9,D0518_Caaguazu_3F,80,46,193,0.238342
